# HW2: Ẩn tin mật trên ảnh gif (ảnh palette-based) bằng phương pháp LSB


Họ tên: Nguyễn Thế Anh

MSSV: 20120248

---

## Nắm cách làm bài và nộp bài

&#9889; Bạn lưu ý là mình sẽ dùng chương trình hỗ trợ chấm bài nên bạn cần phải tuân thủ chính xác qui định mà mình đặt ra, nếu không rõ thì hỏi, chứ không nên tự tiện làm theo ý của cá nhân.

**Cách làm bài**

Bạn sẽ làm trực tiếp trên file notebook này. Đầu tiên, bạn điền họ tên và MSSV vào phần đầu file ở bên trên. Trong file, bạn làm bài ở những chỗ có ghi là:
```python
# YOUR CODE HERE
raise NotImplementedError()
```
hoặc đối với những phần code không bắt buộc thì là:
```python
# YOUR CODE HERE (OPTION)
```
hoặc đối với markdown cell thì là:
```markdown
YOUR ANSWER HERE
```
Tất nhiên, khi làm thì bạn xóa dòng `raise NotImplementedError()` đi.
Đối những phần yêu cầu code thì thường ở ngay phía dưới sẽ có một (hoặc một số) cell chứa các bộ test để phần nào giúp bạn biết đã code đúng hay chưa; nếu chạy cell này không có lỗi gì thì có nghĩa là qua được các bộ test. Trong một số trường hợp, các bộ test có thể sẽ không đầy đủ; nghĩa là, nếu không qua được test thì là code sai, nhưng nếu qua được test thì chưa chắc đã đúng hoàn toàn.

Trong khi làm bài, bạn có thể cho in ra màn hình, tạo thêm các cell để test. Nhưng khi nộp bài thì bạn xóa các cell mà bạn tự tạo, xóa hoặc comment các câu lệnh in ra màn hình. Bạn lưu ý <font color=red>không được tự tiện xóa các cell hay sửa code của Thầy</font> (trừ những chỗ được phép sửa như đã nói ở trên).

Trong khi làm bài, thường xuyên `Ctrl + S` để lưu lại bài làm của bạn, tránh mất mát thông tin.

*Nên nhớ mục tiêu chính ở đây là <font color=green>học, học một cách chân thật</font>.  Bạn có thể thảo luận ý tưởng với bạn khác cũng như tham khảo các nguồn trên mạng, nhưng sau cùng <font color=green>code và bài làm phải là của bạn, dựa trên sự hiểu thật sự của bạn</font> (khi tham khảo các nguồn trên mạng thì bạn cần ghi rõ nguồn trong bài làm, và đương nhiên là bạn cũng không được phép đưa code và bài làm cho bạn khác xem). <font color=red>Nếu vi phạm những điều này thì có thể bạn sẽ bị 0 điểm cho toàn bộ môn học.</font>*

**Cách nộp bài**

Khi chấm bài, đầu tiên mình sẽ chọn `Kernel` - `Restart Kernel & Run All Cells`, để restart và chạy tất cả các cell trong notebook của bạn; do đó, trước khi nộp bài, bạn nên chạy thử `Kernel` - `Restart Kernel & Run All Cells` để đảm bảo mọi chuyện diễn ra đúng như mong đợi.

Sau đó, bạn tạo thư mục nộp bài theo cấu trúc sau:
- Thư mục `MSSV` (vd, nếu bạn có MSSV là 1234567 thì bạn đặt tên thư mục là `1234567`)
    - File `HW2.ipynb` (không cần nộp các file khác)

Cuối cùng, bạn nén thư mục `MSSV` này lại với định dạng nén là .zip (chứ không được là .rar hay các định dạng khác) và nộp ở link trên moodle. \
<font color=red>Bạn lưu ý tuân thủ chính xác qui định nộp bài này.</font>

## Kiểm tra môi trường code 

In [1]:
import sys
sys.executable

'C:\\Python312\\python.exe'

Bạn nên thấy kết quả in ra là đường dẫn đến file chạy python của môi trường "dh_ss-env" mà mình đã hướng dẫn bạn cài đặt ở HW0.

## Import các thư viện cần thiết

In [2]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

## Cài đặt các hàm hỗ trợ

In [3]:
# Ở HW1, ta dùng chuỗi bit (ví dụ: '101') 
# Ở HW này, ta sẽ dùng list các bit (ví dụ: [1, 0, 1]), vì như này sẽ dễ dàng hơn
# cho việc nhúng và rút trích trong HW này
def convert_s2bits(s):
    '''
    Chuyển chuỗi ký tự (ascii) thành list các bit.
    '''
    bits = []
    for c in s:
        c_bits = []
        for bit in list(bin(ord(c))[2:].zfill(8)):
            c_bits.append(int(bit))
        bits.extend(c_bits)
    return bits

def convert_bits2s(bits):
    '''
    Chuyển list các bit thành chuỗi ký tự (ascii).
    '''
    s = ''
    for i in range(0, len(bits), 8):
        c_bits = ''
        for bit in bits[i:i+8]:
            c_bits = c_bits + str(bit)
        c = chr(int(c_bits, 2))
        s = s + c
    return s

In [54]:
# YOUR CODE HERE (OPTION)
def add_padding(msg_arr, num_pixels_cover):
    padding_length = num_pixels_cover - len(msg_arr)
    padded_msg_arr = msg_arr + [1] + [0] * (padding_length - 1)
    return padded_msg_arr

def sub_uint(a, b):
    # if a > b:
    #     return a - b
    # else:
    #     return b - a
    return a - b if a > b else b - a

def get_parity(r, g, b):
    return (r % 2 + g % 2 + b % 2) % 2

# Mảng closest_palette:
# - Mỗi phần tử là index của closest color với màu tương ứng trong bảng palette
# Use to: i_th pixel in stego_img either itself or value of closest_palette[i_th]
def create_closest_palette(palette):
    closest_palette = np.zeros(palette[0].shape[0], dtype=int)
    for i in range(palette[0].shape[0]):
        min_distance = 500
        parity_i = get_parity(palette[0][i][0], palette[0][i][1], palette[0][i][2])
        for j in range(palette[0].shape[0]):
            # No compare with: i_th and same parity in total RGB element
            parity_palette = get_parity(palette[0][j][0], palette[0][j][1], palette[0][j][2])
            if j != i and parity_i != parity_palette:
                p1 = np.array(palette[0][i])
                p2 = np.array(palette[0][j])
                squared_dist = sub_uint(p1[0],p2[0])**2 + sub_uint(p1[1],p2[1])**2 + sub_uint(p1[2],p2[2])**2
                rgb_euclidean_distance = np.sqrt(squared_dist) 
                if rgb_euclidean_distance < min_distance:
                    closest_palette[i] = j
                    min_distance = rgb_euclidean_distance
                    
    return closest_palette
    

## Cài đặt hàm nhúng (7đ, trong đó: phần code 5đ, phần nhận xét và giải thích kết quả 2đ)

&#9889; Lưu ý:
- Khi tính toán bạn cần để ý kiểu dữ liệu. Ví dụ, nếu lấy 2 giá trị có kiểu dữ liệu `uint8` trừ cho nhau thì kết quả cũng sẽ có kiểu `uint8` và sẽ bị sai.
- Qui ước (để mình và bạn ra cùng kết quả): nếu trong bảng màu có nhiều màu thỏa điều kiện tìm kiếm thì lấy màu có chỉ số nhỏ nhất trong bảng màu. Nếu bạn có dùng hàm `sort` hay `argsort` của numpy thì bạn cần truyền tham số `kind="stable"`; nếu không thì các phần tử có giá trị giống nhau có thể sẽ bị thay đổi thứ tự tương đối với nhau trong quá trình sort.

In [55]:
def embed(msg_file, cover_img_file, stego_img_file):
    '''
    Nhúng tin mật vào ảnh gif (ảnh palette-base) bằng phương pháp của Fridrich 
    (xem file slide lý thuyết "05...", trang 45). Khi nhúng thì bạn dùng phương
    pháp thêm đuôi "100..." vào dãy bit của tin mật, và nhúng trên toàn bộ ảnh 
    theo thứ tự từ trái qua phải, từ trên xuống dưới.
    
    Các tham số:
        msg_file (str): Tên file chứa secret message.
        cover_img_file (str): Tên file chứa cover image.
        stego_img_file (str): Tên file chứa stego image (kết quả sau khi nhúng).
    Giá trị trả về:
        bool: True nếu nhúng thành không, False nếu không đủ chỗ để nhúng. 
    '''
    # Đọc cover img file
    cover_img = Image.open(cover_img_file)
    cover_pixels = np.array(cover_img)
    palette = cover_img.getpalette() # Kết quả là list các giá trị Red, Green, Blue của các màu 
                                     # trong bảng màu, 3 giá trị liên tiếp ứng với một màu
    palette = np.array(palette, dtype=np.uint8).reshape(1, -1, 3) # Reshape lại dưới dạng ảnh RGB
    # plt.figure(); plt.yticks([]); plt.imshow(palette, aspect=20) # Uncomment để xem bảng màu
    
    # YOUR CODE HERE
    # Đọc tin mật và chuyển đổi thành chuỗi bit
    with open(msg_file, 'r') as f:
        msg_bytes = f.read()
    msg_arr = convert_s2bits(msg_bytes)
    # Thêm "100..." vào dãy bit nếu cần
    num_pixels_cover = len(cover_pixels) * len(cover_pixels[0])
    if num_pixels_cover > len(msg_arr):
        msg_arr = add_padding(msg_arr, num_pixels_cover)
    else:
        return False

    # Tạo mảng closest_palette, mỗi phần tử là index của closest color với màu tương ứng trong bảng palette
    closest_palette = create_closest_palette(palette)
    
    # Nhúng tin mật vào ảnh
    bit_index = 0
    stego_pixels = np.copy(cover_pixels)
    for i in range(cover_pixels.shape[0]):
        for j in range(cover_pixels.shape[1]):
            # Kiểm tra tính chãn lẻ của pixel so với secret_bit
            color = palette[0][cover_pixels[i, j]]
            parity = get_parity(color[0], color[1], color[2])
            if parity == msg_arr[bit_index]:
                bit_index += 1
                continue
            stego_pixels[i, j] = closest_palette[cover_pixels[i, j]]
            bit_index += 1
    
    # Ghi stego pixels cùng palette xuống file
    stego_img = Image.fromarray(stego_pixels)
    stego_img.putpalette(cover_img.getpalette())
    stego_img.save(stego_img_file)
    return True

In [14]:
# TEST 
result = embed('msg2.txt', 'lena.gif', 'lena_stego.gif')
assert result == False

In [56]:
# TEST             
result = embed('msg.txt', 'lena.gif', 'lena_stego.gif')
assert result == True
stego_img = Image.open('lena_stego.gif')
stego_pixels = np.array(stego_img)
stego_palette = stego_img.getpalette()
correct_stego_img = Image.open('correct_lena_stego.gif')
correct_stego_pixels = np.array(correct_stego_img)
correct_stego_palette = correct_stego_img.getpalette()
assert np.all(stego_pixels == correct_stego_pixels)
assert stego_palette == correct_stego_palette

In [57]:
# TEST             
result = embed('msg.txt', 'baboon.gif', 'baboon_stego.gif')
assert result == True
stego_img = Image.open('baboon_stego.gif')
stego_pixels = np.array(stego_img)
stego_palette = stego_img.getpalette()
correct_stego_img = Image.open('correct_baboon_stego.gif')
correct_stego_pixels = np.array(correct_stego_img)
correct_stego_palette = correct_stego_img.getpalette()
assert np.all(stego_pixels == correct_stego_pixels)
assert stego_palette == correct_stego_palette

Xem các ảnh và so sánh tính vô hình khi nhúng trên ảnh "lena.gif" với khi nhúng trên ảnh "baboon.gif". Theo bạn, tại sao lại như vậy? (Nếu nghĩ mãi mà bạn vẫn không biết tại sao thì bạn cứ nói là không biết tại sao.)

YOUR ANSWER HERE

**So sánh sau khi nhúng trên lena.gif và baboon.gif:**
- Tính vô hình baboon_stego.gif cao hơn hẳn lena_stego.gif
- Khi nhìn bằng mắt thường, baboon_stego.gif dường như y hệt ảnh gốc baboon.gif 
- Còn lena_stego.gif xuất hiện nhiều màu thậm chí là mảng màu dễ dàng nhận biết bằng mắt thường khi so với ảnh gốc lena.gif.

**Giải thích:**
- Lý do chính là sự đa dạng về sắc thái trong bảng màu palette. Ví dụ màu đỏ, bảng palette của baboon.gif có nhiều sắc thái, biến thể từ cam, đỏ nhạt, đỏ đậm,... Do đó, khi chọn 1 màu khác thay thế để nhúng, sự khác nhau sẽ là nhỏ hơn khi so với sự nghèo nàn trong bảng màu ở hình lena.gif.
- Một yếu tố khác là con người có khả năng miễn dịch tốt hơn với việc mất mát các thành phần tần số không gian cao hơn so với việc mất các thành phần tần số thấp hơn. Tức là ảnh baboon.gif có nhiều mảng màu có sự thay đổi liên tục và phong phú về màu sắc hơn so với ảnh lena.gif chỉ tồn tại 1 vài gam màu cơ bản.

## Cài đặt hàm rút trích (3đ)

In [58]:
def extract(stego_img_file, extr_msg_file):
    '''
    Rút trích tin mật đã được nhúng vào ảnh bằng phương pháp của Fridrich 
    (xem file slide lý thuyết "05...", trang 47).
    
    Các tham số:
        stego_img_file (str): Tên file chứa stego image.
        extr_msg_file (str): Tên file chứa secret message được rút trích.
    '''

    # YOUR CODE HERE
    # Đọc stego img file
    stego_img = Image.open(stego_img_file)
    stego_pixels = np.array(stego_img)
    palette = stego_img.getpalette()
    palette = np.array(palette, dtype=np.uint8).reshape(1, -1, 3)
    
    msg_bits = ''.join(str(palette[0, stego_pixels[i, j]].sum() % 2)
                             for i in range(stego_pixels.shape[0])
                             for j in range(stego_pixels.shape[1]))
  
    # bỏ tail 100.. từ extracted_msg_bits
    extracted_msg = convert_bits2s(msg_bits.rstrip("0")[:-1])
    with open(extr_msg_file, 'w') as f:
        f.write(extracted_msg)

In [59]:
# TEST
extract('correct_baboon_stego.gif', 'extr_msg.txt')
with open('extr_msg.txt', 'r') as f:
    extr_msg = f.read()
with open('msg.txt', 'r') as f:
    correct_extr_msg = f.read()
assert extr_msg == correct_extr_msg